In [ ]:
# MTF

## 0. Reveer Ruido
## 1. Generar Patrón
## 1.99 Cargar Patrón
## 2. Inspeccionar Patrón
## 3. Medir

# chusmear: https://www.youtube.com/watch?v=VKYEIxTVCGE

# Estimación de MTF

## Resumen
### 0. Repaso frecuencia y respuesta en frecuencia de sistemas
### 1. Generar patrón
### 2. Adquirir patrón
### 3. Preprocesar muestra
### 4. Extraer MTF
### 5. Para explorar...

In [ ]:
# Cargamos las bibliotecas que usamos casi siempre

import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
print('OpenCV ' + cv2.__version__)
print('numpy ' + np.__version__)

## 0. Repaso frecuencia y respuesta en frecuencia de sistemas + 1. Generar patrón

In [ ]:
# Generar ondas cuadradas para generar el patrón

def generar_onda_cuadrada(largo, periodo):
    # Generar una onda cuadrada con cantdidad de puntos 'largo' y período 'período'.
    # consejo: la función 'mod(a, b)' nos dá el resto de dividir a/b como enteros
    # https://numpy.org/doc/stable/reference/generated/numpy.mod.html
    #


    return onda_cuadrada

plt.plot(generar_onda_cuadrada(largo=100, periodo=14), ':o')


In [ ]:
n_rep = 8
largo_max = 10
ys = []

anchos =       [ 2, 4, 8,16,32,64]
repeticiones = [32,16, 8, 4, 2, 1]
# repeticiones = [ 100, 0, 0, 0, 0, 0]


for a, r in zip(anchos, repeticiones):
    ys.append(generar_onda_cuadrada(a*r, a))

y = np.hstack(ys)
n_samples = len(y)

t = np.linspace(0, n_samples, n_samples)


n_show = n_samples//10
plt.figure(figsize=(10,4))
plt.grid()
# plt.plot(t[:n_show], y[:n_show], ':o')
plt.plot(y, ':o')


In [ ]:
y_f = np.fft.fft(y)

ancho_pantalla = 1920
f = np.linspace(0, 1, n_samples)

plt.figure(figsize=(16,4))
plt.grid()

plt.ylabel('|fft(y)|')
plt.plot(f, np.abs(y_f)/n_samples)
plt.xlabel('f');

x = [1/(a) for a in anchos]
labels = [a for a in anchos]
plt.figure(figsize=(16,4))
plt.ylabel('|fft(y)|')
plt.grid()
n_show = n_samples
plt.plot(f[:n_show], np.abs(y_f[:n_show])/n_samples , ':o')
plt.xticks(x, labels);
plt.xlabel('P');


In [ ]:
## 1. Generar patrón

In [ ]:
# ajustar según pantalla donde se saca la foto, poner la resolución del monitor

# full hd (1080p)
# ancho_pantalla = 1920
# alto_pantalla = 1080

# # típico notebooks:
# ancho_pantalla = 1368
# alto_pantalla = 768

# alguien tiene pantalla "retina" o 4k?
ancho_pantalla = 
alto_pantalla = 

alto_franja_central_px = 40

oscuro = 64
claro = 192

medio = (claro+oscuro)/2
amplitud = (claro-oscuro)/2

onda = medio + amplitud*y

centro_izq = 255*np.ones((alto_franja_central_px, ancho_pantalla//2))
centro_der =   0*np.ones((alto_franja_central_px, ancho_pantalla//2 - len(onda)))

arriba = claro*np.ones((alto_pantalla//2 - alto_franja_central_px//2, ancho_pantalla))
centro = np.hstack((centro_izq, np.tile(onda, (alto_franja_central_px, 1)), centro_der))
abajo = oscuro*np.ones((alto_pantalla//2 - alto_franja_central_px//2, ancho_pantalla))

patron_mtf = np.vstack((arriba, centro, abajo)).astype(np.uint8)

patron_mtf = cv2.cvtColor(patron_mtf, cv2.COLOR_GRAY2RGB)

plt.imshow(patron_mtf)
cv2.imwrite('patron_mtf.png', patron_mtf)

![title](patron_mtf.png)

## 2. Adquirir patrón

Poner la imagen que generaron a pantalla completa y sacar foto.
Consejos para la foto:
1. en condiciones de bastante luz ambiente así no sale "quemada"
2. distancia como para que el teléfono no muestre detalle de los píxeles
3. horizontal y centrada en el patrón
4. sacar varias de una y transferirlas de lo posible no via Google Fotos o Whatsapp o equivalente para evitar distorsiones por compresión (de lo posible conectar el celular o tarjeta de memoria a la compu).

In [ ]:
mtf_leer = cv2.imread('./fotos/mtf.jpg')

# pasamos a gris
mtf_leido_gray = cv2.cvtColor(mtf_leer, cv2.COLOR_BGR2GRAY)

plt.imshow(mtf_leido_gray, cmap='gray')

## 3. Preprocesar muestra

In [ ]:
# Buscar de pegarle con un corte a la zona del mtf
ajustar_linea_mtf = cv2.cvtColor(mtf_leido_gray//4, cv2.COLOR_GRAY2RGB)

ancho_foto = mtf_leido_gray.shape[1]

fila_mtf = 1500
zoom = 1600

# rellenar con principio y fin del patrón mtf en la foto:
columnas_mtf = slice( , )

ajustar_linea_mtf[fila_mtf, columnas_mtf, 0] = 255

plt.figure(figsize=(16,9));
plt.imshow(ajustar_linea_mtf);


In [ ]:
# generar algunos trazados para ver qué hay en esa línea
plt.figure(figsize=(10,4))
plt.plot(mtf_leido_gray[fila_mtf,:])

zoom = 1000
plt.figure(figsize=(10,4))
plt.plot(mtf_leido_gray[fila_mtf, columnas_mtf])

plt.figure(figsize=(10,4))
zoom = 100
plt.plot(mtf_leido_gray[fila_mtf, columnas_mtf])

In [ ]:
# refinamos el corte
zoom = 1000
fila_mtf = 1500

columnas_mtf = slice((ancho_foto//2-zoom//2), (ancho_foto//2+zoom//2))


In [ ]:
# dibujamos con más detalle el corte

y_est = mtf_leido_gray[fila_mtf, columnas_mtf]

plt.figure(figsize=(10,4))
plt.plot(y_est)

In [ ]:
# Generamos gráfico para leer donde cortar para recuperar la estimación de la y de arriba

# por ahí si no quedó perfectamente centrado necesitan otro valor diferente de +/- 35:
claro_est = mtf_leido_gray[fila_mtf-35, columnas_mtf]
oscuro_est = mtf_leido_gray[fila_mtf+35, columnas_mtf]

%matplotlib notebook
plt.figure(figsize=(10,4))
plt.plot(y_est)
plt.plot(claro_est)
plt.plot(oscuro_est)

In [ ]:
# Leer del gráfico de arriba

k_start =  
k_end = 

y_est_r = y_est[k_start:k_end] 

%matplotlib notebook
plt.figure(figsize=(10,4))
plt.grid()
plt.plot(y_est_r)



In [ ]:
# llenar con bordes medidos del gráfico:
bordes= [ ......
         
%matplotlib inline
plt.figure(figsize=(10,4))
plt.grid()
plt.plot(y_est_r)

for b in bordes:
    plt.plot([b, b], [0, 255], ':k')


## 4. Extraer MTF

In [ ]:
# comparamos con la función que le "metimos de entrada" al sistema
%matplotlib inline
plt.figure(figsize=(16,4))
plt.grid()
plt.plot(y_est_r)

# -9,455 son offsets para que quede bien, cambiarlo según lo que obtuvieron
offset_l = -9
offset_r = 455

x_est=np.linspace(offset_l,offset_r,len(onda))
plt.plot(x_est, onda, ':*')

In [ ]:
# idem normalizado
%matplotlib inline
plt.figure(figsize=(16,4))
plt.grid()
plt.plot((y_est_r-np.min(y_est_r))/(np.max(y_est_r)-np.min(y_est_r)))
x_est=np.linspace(offset_l,offset_r,len(onda))
plt.plot(x_est, (onda-np.min(onda))/(np.max(onda)-np.min(onda)), ':*')

In [ ]:
# Sacar fotos y tomar nota de la geometría de cómo tomaron la foto en particular distancia al monitor 
# y ángulo subtendido por zona usada para calcular mtf:
distancia_monitor_mm = 20.000km
ancho_zona_mtf_mm = 1nm

# Calcular ángulo que genera la zona usada para medir mtf (cateto menor en la pantalla 
# y cateto mayor distancia entre cámara y pantalla)
angulo_zona_mtf_deg = np.arctan2(ancho_zona_mtf_mm, distancia_monitor_mm)*180/np.pi
ancho_zona_mtf_px = len(y_est_r)





In [ ]:
# Calculamos grados por pixel para usar en el gráfico
deg_per_px = ancho_zona_mtf_px/angulo_zona_mtf_deg


$ \text{MTF}_{\text{Local}} = \dfrac{i_{max}-i_{min}}{i_{max}+i_{min}}$

In [ ]:
# medimos el contraste en cada tramo y lo graficamos
tramos = np.split(y_est_r, bordes)

mtfs = []

for tramo in tramos:
    i_max = máximo en el tramo
    i_min = mínimo en el tramo
    mtf = fórmula de mtf como relación de intensidades
    mtfs.append(mtf)
    print(mtf)
    
plt.grid(); plt.title('MTF')
plt.plot(1/np.array(anchos)*deg_per_px, mtfs, ':o')
plt.ylabel('MTF');
plt.xlabel('lp/deg');

## 5. Propuestas para explorar:

1. Probar con otros juegos de barras
2. Detectar automáticamente dónde está la zona para sacar MTF y conociendo las barras calcularlo automáticamente
3. Probar con fotos de cerca y de lejos y en distintas condiciones de iluminación
4. Probar con otros teléfonos o cámaras (por ejemplo cámara frontal de selfie)
5. (Ir a algoritmos más avanzados (fft, slanted edge))